In [2]:
import pandas as pd
meta = pd.read_csv('AnonDB.csv')

In [1]:
import numpy as np
import pandas as pd

# -----------------------------
# 1) Solver I(V) untuk SDM (implicit)
# -----------------------------
def solve_current_sdm(V, params, max_iter=80, tol=1e-10):
    """
    Solve I for a given V using Newton-Raphson.
    params: dict with keys: Ipv, I0, Rs, Rp, a, Vt
    """
    Ipv, I0, Rs, Rp, a, Vt = (
        params["Ipv"], params["I0"], params["Rs"], params["Rp"], params["a"], params["Vt"]
    )

    # Initial guess: close to short-circuit current
    I = np.clip(Ipv, 0, None)

    for _ in range(max_iter):
        x = (V + I * Rs) / (a * Vt)

        # avoid overflow
        x = np.clip(x, -100, 100)
        ex = np.exp(x)

        # f(I) = I - Ipv + I0*(exp(x)-1) + (V + I*Rs)/Rp = 0
        f = I - Ipv + I0 * (ex - 1.0) + (V + I * Rs) / Rp

        # df/dI = 1 + I0*exp(x) * (Rs/(a*Vt)) + Rs/Rp
        df = 1.0 + I0 * ex * (Rs / (a * Vt)) + (Rs / Rp)

        step = f / df
        I_new = I - step

        if np.abs(I_new - I) < tol:
            return float(I_new)

        I = I_new

    return float(I)  # return best effort


def iv_curve(params, V_grid):
    I = np.array([solve_current_sdm(V, params) for V in V_grid], dtype=float)
    P = V_grid * I
    return I, P


def calc_pmpp(params, Voc_guess=0.72, nV=200):
    """
    Approximate Pmpp by scanning V from 0 to Voc_guess.
    For better accuracy, set Voc_guess based on your module/string.
    """
    V_grid = np.linspace(0, Voc_guess, nV)
    I, P = iv_curve(params, V_grid)
    idx = int(np.nanargmax(P))
    return float(P[idx]), float(V_grid[idx]), float(I[idx])


# -----------------------------
# 2) Sampling distributions (Monte Carlo)
# -----------------------------
def sample_lognormal(mean, cov, size, rng):
    """
    lognormal with desired mean and coefficient of variation (std/mean).
    Works for strictly positive parameters.
    """
    mean = float(mean)
    cov = float(cov)
    # convert (mean, cov) -> lognormal mu, sigma
    sigma2 = np.log(1 + cov**2)
    sigma = np.sqrt(sigma2)
    mu = np.log(mean) - 0.5 * sigma2
    return rng.lognormal(mean=mu, sigma=sigma, size=size)


def sample_normal(mean, std, size, rng):
    return rng.normal(loc=mean, scale=std, size=size)


# -----------------------------
# 3) Rank + PRCC utilities
# -----------------------------
def rankdata(a):
    # simple rank (ties handled approximately by average rank via pandas)
    return pd.Series(a).rank(method="average").to_numpy()

def prcc(X, y):
    """
    Partial Rank Correlation Coefficient:
    - rank-transform all variables
    - regress y on other X (excluding Xi), take residuals
    - regress Xi on other X, take residuals
    - corr(res_y, res_xi)
    """
    Xr = np.column_stack([rankdata(X[:, j]) for j in range(X.shape[1])])
    yr = rankdata(y)

    # add intercept
    ones = np.ones((Xr.shape[0], 1))
    PRCC = np.zeros(Xr.shape[1], dtype=float)

    for i in range(Xr.shape[1]):
        idx_other = [j for j in range(Xr.shape[1]) if j != i]
        X_other = np.column_stack([ones, Xr[:, idx_other]])

        # residual of y ~ other
        beta_y, *_ = np.linalg.lstsq(X_other, yr, rcond=None)
        y_hat = X_other @ beta_y
        ry = yr - y_hat

        # residual of Xi ~ other
        beta_x, *_ = np.linalg.lstsq(X_other, Xr[:, i], rcond=None)
        x_hat = X_other @ beta_x
        rx = Xr[:, i] - x_hat

        # correlation
        PRCC[i] = np.corrcoef(rx, ry)[0, 1]

    return PRCC


# -----------------------------
# 4) Main Monte Carlo simulation
# -----------------------------
def monte_carlo_sensitivity(
    N=3000,
    seed=42,
    V_eval=0.55,          # evaluate current at a specific voltage (example)
    Voc_guess=0.72,       # for Pmpp scanning
):
    rng = np.random.default_rng(seed)

    # ---- Nominal parameters (EDIT sesuai modulmu) ----
    # Ipv (A), I0 (A), Rs (ohm), Rp (ohm), a (-), Vt (V)
    # NOTE: Vt = (kT/q) * Ns, jadi tergantung jumlah sel seri & temperatur
    nominal = dict(
        Ipv=9.0,
        I0=1e-10,
        Rs=0.35,
        Rp=800.0,
        a=1.2,
        Vt=0.026 * 60,   # contoh: 60 sel seri @ ~300K => 1.56 V
    )

    # ---- Uncertainty assumptions (EDIT) ----
    # Positive parameters -> lognormal (more realistic)
    cov = dict(
        Ipv=0.03,   # 3%
        I0=0.80,    # I0 sering sangat variatif
        Rs=0.15,
        Rp=0.30,
        a=0.05,
        Vt=0.02,
    )

    # Sample parameters
    samples = pd.DataFrame({
        "Ipv": sample_lognormal(nominal["Ipv"], cov["Ipv"], N, rng),
        "I0":  sample_lognormal(nominal["I0"],  cov["I0"],  N, rng),
        "Rs":  sample_lognormal(nominal["Rs"],  cov["Rs"],  N, rng),
        "Rp":  sample_lognormal(nominal["Rp"],  cov["Rp"],  N, rng),
        "a":   sample_lognormal(nominal["a"],   cov["a"],   N, rng),
        "Vt":  sample_lognormal(nominal["Vt"],  cov["Vt"],  N, rng),
    })

    # Outputs
    I_at_V = np.empty(N, dtype=float)
    Pmpp = np.empty(N, dtype=float)

    for i in range(N):
        p = samples.iloc[i].to_dict()
        # output 1: current at V_eval
        I_at_V[i] = solve_current_sdm(V_eval, p)

        # output 2: Pmpp (approx) from IV scanning
        Pmpp[i], _, _ = calc_pmpp(p, Voc_guess=Voc_guess, nV=220)

    # Build X matrix
    X = samples.to_numpy()
    param_names = samples.columns.tolist()

    # Spearman correlations (rank corr)
    spearman_I = [np.corrcoef(rankdata(X[:, j]), rankdata(I_at_V))[0, 1] for j in range(X.shape[1])]
    spearman_P = [np.corrcoef(rankdata(X[:, j]), rankdata(Pmpp))[0, 1] for j in range(X.shape[1])]

    # PRCC
    prcc_I = prcc(X, I_at_V)
    prcc_P = prcc(X, Pmpp)

    # Summary table
    out = pd.DataFrame({
        "parameter": param_names,
        "Spearman(I@V_eval)": spearman_I,
        "PRCC(I@V_eval)": prcc_I,
        "Spearman(Pmpp)": spearman_P,
        "PRCC(Pmpp)": prcc_P,
        "abs_PRCC(Pmpp)": np.abs(prcc_P),
        "abs_PRCC(I@V_eval)": np.abs(prcc_I),
    }).sort_values("abs_PRCC(Pmpp)", ascending=False)

    # Add basic distribution info
    desc = samples.describe(percentiles=[0.05, 0.5, 0.95]).T[["mean", "std", "5%", "50%", "95%"]]
    return out.reset_index(drop=True), desc, I_at_V, Pmpp, samples


if __name__ == "__main__":
    ranking, param_stats, I_at_V, Pmpp, samples = monte_carlo_sensitivity(
        N=3000,
        seed=1,
        V_eval=0.55,
        Voc_guess=0.72,
    )

    print("\n=== Parameter stats (sampled) ===")
    print(param_stats)

    print("\n=== Sensitivity ranking (higher |PRCC| => more influential) ===")
    print(ranking[["parameter", "PRCC(Pmpp)", "Spearman(Pmpp)", "PRCC(I@V_eval)", "Spearman(I@V_eval)"]])

    print("\nPmpp summary:", pd.Series(Pmpp).describe())
    print("I(V_eval) summary:", pd.Series(I_at_V).describe())



=== Parameter stats (sampled) ===
             mean           std            5%           50%           95%
Ipv  8.999329e+00  2.688421e-01  8.568111e+00  8.995682e+00  9.451849e+00
I0   9.975245e-11  8.450028e-11  2.447156e-11  7.682395e-11  2.426250e-10
Rs   3.498114e-01  5.215127e-02  2.722976e-01  3.461650e-01  4.414029e-01
Rp   7.922339e+02  2.335495e+02  4.711153e+02  7.622026e+02  1.219899e+03
a    1.198938e+00  6.001090e-02  1.105873e+00  1.196331e+00  1.302274e+00
Vt   1.559951e+00  3.083508e-02  1.510161e+00  1.559883e+00  1.611433e+00

=== Sensitivity ranking (higher |PRCC| => more influential) ===
  parameter  PRCC(Pmpp)  Spearman(Pmpp)  PRCC(I@V_eval)  Spearman(I@V_eval)
0       Ipv    0.999991        0.999973        0.999992            0.999974
1        Rp    0.797269        0.033640        0.791606            0.033378
2        Rs   -0.488618       -0.006006       -0.499434           -0.006006
3        I0   -0.036272        0.003335       -0.038014            0.003328
4 

In [2]:
import numpy as np
import pandas as pd

# -----------------------------
# 1) Solver I(V) untuk Double-Diode Model (implicit)
# -----------------------------
def solve_current_ddm(V, params, max_iter=100, tol=1e-11):
    """
    Double diode model:
    I = Ipv - I01*(exp((V+I*Rs)/(a1*Vt1)) - 1)
            - I02*(exp((V+I*Rs)/(a2*Vt2)) - 1)
            - (V + I*Rs)/Rp
    Solve I for a given V using Newton-Raphson.
    """
    Ipv = params["Ipv"]
    I01 = params["I01"]
    I02 = params["I02"]
    Rs  = params["Rs"]
    Rp  = params["Rp"]
    a1  = params["a1"]
    a2  = params["a2"]
    Vt1 = params["Vt1"]
    Vt2 = params["Vt2"]

    # initial guess: near short-circuit current
    I = np.clip(Ipv, 0, None)

    for _ in range(max_iter):
        x1 = (V + I * Rs) / (a1 * Vt1)
        x2 = (V + I * Rs) / (a2 * Vt2)

        # prevent overflow
        x1 = np.clip(x1, -100, 100)
        x2 = np.clip(x2, -100, 100)

        ex1 = np.exp(x1)
        ex2 = np.exp(x2)

        # f(I) = I - Ipv + I01*(exp(x1)-1) + I02*(exp(x2)-1) + (V+I*Rs)/Rp = 0
        f = I - Ipv + I01 * (ex1 - 1.0) + I02 * (ex2 - 1.0) + (V + I * Rs) / Rp

        # df/dI = 1 + I01*exp(x1)*(Rs/(a1*Vt1)) + I02*exp(x2)*(Rs/(a2*Vt2)) + Rs/Rp
        df = (
            1.0
            + I01 * ex1 * (Rs / (a1 * Vt1))
            + I02 * ex2 * (Rs / (a2 * Vt2))
            + (Rs / Rp)
        )

        step = f / df
        I_new = I - step

        if np.abs(I_new - I) < tol:
            return float(I_new)

        I = I_new

    return float(I)


def iv_curve_ddm(params, V_grid):
    I = np.array([solve_current_ddm(V, params) for V in V_grid], dtype=float)
    P = V_grid * I
    return I, P


def calc_pmpp_ddm(params, Voc_guess=0.72, nV=220):
    V_grid = np.linspace(0, Voc_guess, nV)
    I, P = iv_curve_ddm(params, V_grid)
    idx = int(np.nanargmax(P))
    return float(P[idx]), float(V_grid[idx]), float(I[idx])


# -----------------------------
# 2) Sampling distributions (Monte Carlo)
# -----------------------------
def sample_lognormal(mean, cov, size, rng):
    mean = float(mean)
    cov = float(cov)
    sigma2 = np.log(1 + cov**2)
    sigma = np.sqrt(sigma2)
    mu = np.log(mean) - 0.5 * sigma2
    return rng.lognormal(mean=mu, sigma=sigma, size=size)


def rankdata(a):
    return pd.Series(a).rank(method="average").to_numpy()


def prcc(X, y):
    Xr = np.column_stack([rankdata(X[:, j]) for j in range(X.shape[1])])
    yr = rankdata(y)

    ones = np.ones((Xr.shape[0], 1))
    PRCC = np.zeros(Xr.shape[1], dtype=float)

    for i in range(Xr.shape[1]):
        idx_other = [j for j in range(Xr.shape[1]) if j != i]
        X_other = np.column_stack([ones, Xr[:, idx_other]])

        beta_y, *_ = np.linalg.lstsq(X_other, yr, rcond=None)
        ry = yr - (X_other @ beta_y)

        beta_x, *_ = np.linalg.lstsq(X_other, Xr[:, i], rcond=None)
        rx = Xr[:, i] - (X_other @ beta_x)

        PRCC[i] = np.corrcoef(rx, ry)[0, 1]

    return PRCC


# -----------------------------
# 3) Main Monte Carlo sensitivity for DDM
# -----------------------------
def monte_carlo_sensitivity_ddm(
    N=3000,
    seed=1,
    V_eval=0.55,
    Voc_guess=0.72,
):
    rng = np.random.default_rng(seed)

    # ---- Nominal parameters (EDIT sesuai modul/dataset kamu) ----
    # Tipikal: I01 << I02 (atau sebaliknya tergantung model fit), dan I0 bisa sangat kecil
    nominal = dict(
        Ipv=9.0,
        I01=1e-12,
        I02=1e-8,
        Rs=0.35,
        Rp=800.0,
        a1=1.2,
        a2=2.0,
        Vt1=0.026 * 60,   # Ns*Vt_cell @ ~300K
        Vt2=0.026 * 60,
    )

    # ---- Uncertainty assumptions (EDIT) ----
    cov = dict(
        Ipv=0.03,
        I01=1.00,   # sangat variatif
        I02=0.80,
        Rs=0.15,
        Rp=0.30,
        a1=0.05,
        a2=0.05,
        Vt1=0.02,
        Vt2=0.02,
    )

    # Sample parameters (positive -> lognormal)
    samples = pd.DataFrame({
        "Ipv": sample_lognormal(nominal["Ipv"], cov["Ipv"], N, rng),
        "I01": sample_lognormal(nominal["I01"], cov["I01"], N, rng),
        "I02": sample_lognormal(nominal["I02"], cov["I02"], N, rng),
        "Rs":  sample_lognormal(nominal["Rs"],  cov["Rs"],  N, rng),
        "Rp":  sample_lognormal(nominal["Rp"],  cov["Rp"],  N, rng),
        "a1":  sample_lognormal(nominal["a1"],  cov["a1"],  N, rng),
        "a2":  sample_lognormal(nominal["a2"],  cov["a2"],  N, rng),
        "Vt1": sample_lognormal(nominal["Vt1"], cov["Vt1"], N, rng),
        "Vt2": sample_lognormal(nominal["Vt2"], cov["Vt2"], N, rng),
    })

    # Outputs
    I_at_V = np.empty(N, dtype=float)
    Pmpp   = np.empty(N, dtype=float)

    for i in range(N):
        p = samples.iloc[i].to_dict()
        I_at_V[i] = solve_current_ddm(V_eval, p)
        Pmpp[i], _, _ = calc_pmpp_ddm(p, Voc_guess=Voc_guess, nV=240)

    X = samples.to_numpy()
    param_names = samples.columns.tolist()

    # Spearman
    spearman_I = [np.corrcoef(rankdata(X[:, j]), rankdata(I_at_V))[0, 1] for j in range(X.shape[1])]
    spearman_P = [np.corrcoef(rankdata(X[:, j]), rankdata(Pmpp))[0, 1] for j in range(X.shape[1])]

    # PRCC
    prcc_I = prcc(X, I_at_V)
    prcc_P = prcc(X, Pmpp)

    ranking = pd.DataFrame({
        "parameter": param_names,
        "Spearman(I@V_eval)": spearman_I,
        "PRCC(I@V_eval)": prcc_I,
        "Spearman(Pmpp)": spearman_P,
        "PRCC(Pmpp)": prcc_P,
        "abs_PRCC(Pmpp)": np.abs(prcc_P),
    }).sort_values("abs_PRCC(Pmpp)", ascending=False).reset_index(drop=True)

    param_stats = samples.describe(percentiles=[0.05, 0.5, 0.95]).T[["mean", "std", "5%", "50%", "95%"]]

    return ranking, param_stats, I_at_V, Pmpp, samples


if __name__ == "__main__":
    ranking, param_stats, I_at_V, Pmpp, samples = monte_carlo_sensitivity_ddm(
        N=3000,
        seed=1,
        V_eval=0.55,
        Voc_guess=0.72,
    )

    print("\n=== Parameter stats (sampled) ===")
    print(param_stats)

    print("\n=== Sensitivity ranking (sorted by |PRCC| on Pmpp) ===")
    print(ranking[["parameter", "PRCC(Pmpp)", "Spearman(Pmpp)", "PRCC(I@V_eval)", "Spearman(I@V_eval)"]])

    print("\nPmpp summary:\n", pd.Series(Pmpp).describe())
    print("\nI(V_eval) summary:\n", pd.Series(I_at_V).describe())



=== Parameter stats (sampled) ===
             mean           std            5%           50%           95%
Ipv  8.999329e+00  2.688421e-01  8.568111e+00  8.995682e+00  9.451849e+00
I01  1.000214e-12  1.082598e-12  1.790592e-13  6.935898e-13  2.705770e-12
I02  9.951594e-09  7.964748e-09  2.519365e-09  7.812658e-09  2.457510e-08
Rs   3.483807e-01  5.147520e-02  2.703301e-01  3.451950e-01  4.383808e-01
Rp   7.961015e+02  2.421510e+02  4.776887e+02  7.581389e+02  1.248085e+03
a1   1.199884e+00  5.929598e-02  1.105659e+00  1.198876e+00  1.300339e+00
a2   1.999032e+00  9.885115e-02  1.839057e+00  1.998528e+00  2.171442e+00
Vt1  1.560024e+00  3.056834e-02  1.510718e+00  1.559143e+00  1.611664e+00
Vt2  1.559857e+00  3.101535e-02  1.510423e+00  1.559115e+00  1.611607e+00

=== Sensitivity ranking (sorted by |PRCC| on Pmpp) ===
  parameter  PRCC(Pmpp)  Spearman(Pmpp)  PRCC(I@V_eval)  Spearman(I@V_eval)
0       Ipv    0.999992        0.999975        0.999992            0.999977
1        Rp    0.

In [ ]:
import numpy as np


# =========================================================
# Helper
# =========================================================

def normalize_0_1(img):
    """
    Normalize grayscale image to range [0, 1].
    """
    img = img.astype(np.float32)
    return (img - img.min()) / (img.max() - img.min() + 1e-9)


def coefficient_of_variation(img):
    """
    CV = std / mean
    """
    return img.std() / (img.mean() + 1e-9)


# =========================================================
# 1) Iph from active area (EL 80%)
# =========================================================

def estimate_Iph(el80, Iph_ref=9.0, threshold=0.35):
    """
    Iph proportional to active area.
    """
    el80_n = normalize_0_1(el80)

    active_pixels = el80_n > threshold
    active_ratio = active_pixels.mean()

    Iph = Iph_ref * active_ratio
    return Iph, active_ratio


# =========================================================
# 2) Rs from EL 80% non-uniformity
# =========================================================

def estimate_Rs(el80, Rs_ref=0.35, alpha=1.0):
    """
    Rs increases with EL intensity non-uniformity.
    """
    el80_n = normalize_0_1(el80)

    cv = coefficient_of_variation(el80_n)
    Rs = Rs_ref * (1.0 + alpha * cv)

    return Rs, cv


# =========================================================
# 3) Rp from EL 20% uniformity (shunt proxy)
# =========================================================

def estimate_Rp(el20, Rp_ref=800.0, beta=1.0):
    """
    Rp increases with EL uniformity at low current.
    """
    el20_n = normalize_0_1(el20)

    mean_intensity = el20_n.mean()
    std_intensity = el20_n.std()

    uniformity = mean_intensity / (std_intensity + 1e-9)
    Rp = Rp_ref * (1.0 + beta * uniformity)

    return Rp, uniformity


# =========================================================
# 4) All-in-one wrapper
# =========================================================

def estimate_parameters(el80, el20,
                        Iph_ref=9.0,
                        Rs_ref=0.35,
                        Rp_ref=800.0,
                        threshold=0.35,
                        alpha=1.0,
                        beta=1.0):
    """
    Estimate Iph, Rs, Rp from grayscale EL images.
    """

    Iph, active_ratio = estimate_Iph(el80, Iph_ref, threshold)
    Rs, cv_80 = estimate_Rs(el80, Rs_ref, alpha)
    Rp, uniformity_20 = estimate_Rp(el20, Rp_ref, beta)

    return {
        "Iph": Iph,
        "Rs": Rs,
        "Rp": Rp,
        "active_ratio": active_ratio,
        "cv_80": cv_80,
        "uniformity_20": uniformity_20
    }


# =========================================================
# Example usage
# =========================================================
if __name__ == "__main__":
    el80 = np.load("el_80.npy")   # or cv2.imread(..., IMREAD_GRAYSCALE)
    el20 = np.load("el_20.npy")

    params = estimate_parameters(
        el80, el20,
        Iph_ref=9.0,
        Rs_ref=0.35,
        Rp_ref=800.0,
        threshold=0.35,
        alpha=1.0,
        beta=1.0
    )

    for k, v in params.items():
        print(f"{k:>15}: {v:.4f}")


In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import cv2


# ==============================
# CONFIG
# ==============================
INPUT_DIR = r"C:\Users\Ghozy Abror\OneDrive - Institut Teknologi Bandung\Karirku\UNSW\Thesis\Coding\EL\EL_wrp"   # <-- ganti folder kamu
REMOVE_VALUES = {0, 1}                       # nilai yang mau dieliminasi
SAVE_CSV = True
OUTPUT_CSV = "pixel_stats.csv"


# ==============================
# CORE FUNCTIONS
# ==============================
def read_grayscale_tiff(path: Path) -> np.ndarray:
    """
    Read TIFF as grayscale (uint8). Assumes image is already 8-bit grayscale.
    """
    img = cv2.imread(str(path), cv2.IMREAD_UNCHANGED)
    if img is None:
        raise ValueError(f"Cannot read image: {path}")

    # If somehow read as multi-channel, convert to grayscale
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Ensure uint8
    if img.dtype != np.uint8:
        # still handle safely
        img = img.astype(np.uint8)

    return img


def remove_values(img: np.ndarray, values_to_remove={0, 1}) -> np.ndarray:
    """
    Return 1D array of pixels after removing certain values (e.g., 0 and 1).
    """
    mask = np.ones_like(img, dtype=bool)
    for v in values_to_remove:
        mask &= (img != v)
    return img[mask].ravel()


def compute_stats(pixels_1d: np.ndarray):
    """
    Compute mean, median, mode for uint8 pixel values.
    Mode computed via bincount (fast for 0..255).
    """
    if pixels_1d.size == 0:
        return np.nan, np.nan, np.nan

    mean_val = float(pixels_1d.mean())
    median_val = float(np.median(pixels_1d))

    # Mode (most frequent value)
    counts = np.bincount(pixels_1d, minlength=256)  # because uint8
    mode_val = int(np.argmax(counts))

    return mean_val, median_val, mode_val


# ==============================
# MAIN PIPELINE
# ==============================
def build_pixel_stats_table(input_dir: str, remove_vals={0, 1}) -> pd.DataFrame:
    folder = Path(input_dir)
    if not folder.exists():
        raise FileNotFoundError(f"Folder not found: {folder}")

    # collect tif/tiff
    image_paths = sorted(list(folder.glob("*.tif")) + list(folder.glob("*.tiff")))

    rows = []
    for p in image_paths:
        img = read_grayscale_tiff(p)
        pixels = remove_values(img, remove_vals)
        mean_val, median_val, mode_val = compute_stats(pixels)

        rows.append({
            "nama_gambar": p.name,
            "mean": mean_val,
            "median": median_val,
            "modus": mode_val
        })

    return pd.DataFrame(rows)


if __name__ == "__main__":
    df = build_pixel_stats_table(INPUT_DIR, REMOVE_VALUES)
    print(df)

    if SAVE_CSV:
        df.to_csv(OUTPUT_CSV, index=False)
        print(f"\nSaved: {OUTPUT_CSV}")


                          nama_gambar       mean  median  modus
0     10084_35_1_01272020_20_wrp.tiff  89.951571    95.0    102
1     10084_35_1_01272020_80_wrp.tiff  67.735187    70.0     70
2     10084_35_1_02222021_20_wrp.tiff  88.351796   105.0      5
3     10084_35_1_02222021_80_wrp.tiff  72.874862    85.0      6
4     10085_35_1_01272020_20_wrp.tiff  82.821786    86.0     77
...                               ...        ...     ...    ...
1191   4856_32_2_11022021_80_wrp.tiff  88.378453    93.0     12
1192   4857_32_2_11022021_20_wrp.tiff  89.341477    96.0     12
1193   4857_32_2_11022021_80_wrp.tiff  88.050140    94.0     12
1194   4896_32_2_10262021_20_wrp.tiff  99.791134   109.0     13
1195   4896_32_2_10262021_80_wrp.tiff  91.201926    98.0     12

[1196 rows x 4 columns]

Saved: pixel_stats.csv


In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import cv2


INPUT_DIR = r"C:\Users\Ghozy Abror\OneDrive - Institut Teknologi Bandung\Karirku\UNSW\Thesis\Coding\EL\EL_wrp"   # <-- ganti folder kamu
REMOVE_VALUES = {0, 1}


def read_grayscale_tiff(path: Path):
    img = cv2.imread(str(path), cv2.IMREAD_UNCHANGED)

    if img is None:
        raise ValueError(f"Cannot read image: {path}")

    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    return img.astype(np.uint8)


def remove_values(img, values_to_remove={0, 1}):
    mask = np.ones_like(img, dtype=bool)
    for v in values_to_remove:
        mask &= (img != v)
    return img[mask].ravel()


def compute_stats(pixels_1d, panel_max):
    if pixels_1d.size == 0:
        return np.nan, np.nan, np.nan, np.nan

    mean_val = float(pixels_1d.mean())
    median_val = float(np.median(pixels_1d))

    counts = np.bincount(pixels_1d, minlength=256)
    mode_val = int(np.argmax(counts))

    # Intensitas relatif terhadap pixel paling terang di panel
    mean_relative = mean_val / (panel_max + 1e-9)

    return mean_val, median_val, mode_val, mean_relative


def build_pixel_stats_table(input_dir, remove_vals={0, 1}):
    folder = Path(input_dir)
    image_paths = sorted(list(folder.glob("*.tif")) +
                         list(folder.glob("*.tiff")))

    rows = []

    for p in image_paths:
        img = read_grayscale_tiff(p)

        panel_max = img.max()   # pixel paling terang di panel
        pixels = remove_values(img, remove_vals)

        mean_val, median_val, mode_val, mean_rel = compute_stats(
            pixels, panel_max
        )

        rows.append({
            "nama_gambar": p.name,
            "mean": mean_val,
            "median": median_val,
            "modus": mode_val,
            "mean_relative": mean_rel
        })

    return pd.DataFrame(rows)


if __name__ == "__main__":
    df = build_pixel_stats_table(INPUT_DIR, REMOVE_VALUES)
    print(df)


                          nama_gambar       mean  median  modus  mean_relative
0     10084_35_1_01272020_20_wrp.tiff  89.951571    95.0    102       0.473429
1     10084_35_1_01272020_80_wrp.tiff  67.735187    70.0     70       0.378409
2     10084_35_1_02222021_20_wrp.tiff  88.351796   105.0      5       0.450774
3     10084_35_1_02222021_80_wrp.tiff  72.874862    85.0      6       0.385581
4     10085_35_1_01272020_20_wrp.tiff  82.821786    86.0     77       0.452578
...                               ...        ...     ...    ...            ...
1191   4856_32_2_11022021_80_wrp.tiff  88.378453    93.0     12       0.392793
1192   4857_32_2_11022021_20_wrp.tiff  89.341477    96.0     12       0.446707
1193   4857_32_2_11022021_80_wrp.tiff  88.050140    94.0     12       0.421293
1194   4896_32_2_10262021_20_wrp.tiff  99.791134   109.0     13       0.498956
1195   4896_32_2_10262021_80_wrp.tiff  91.201926    98.0     12       0.357655

[1196 rows x 5 columns]


In [2]:
df.to_csv("pixel_stats_2.csv", index=False)